In [2]:
from google.colab import  drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import pandas as pd
df = pd.read_csv('/content/gdrive/My Drive/Machine Learning/Categorical Encoder practice/application_train.csv')


# (A) ONE-HOT ENCODING

get DUMMIES for One-Hot

In [4]:

df = df[['OCCUPATION_TYPE','OWN_CAR_AGE','TARGET']]
df['OCCUPATION_TYPE'].value_counts(dropna=False)


NaN                      96391
Laborers                 55186
Sales staff              32102
Core staff               27570
Managers                 21371
Drivers                  18603
High skill tech staff    11380
Accountants               9813
Medicine staff            8537
Security staff            6721
Cooking staff             5946
Cleaning staff            4653
Private service staff     2652
Low-skill Laborers        2093
Waiters/barmen staff      1348
Secretaries               1305
Realty agents              751
HR staff                   563
IT staff                   526
Name: OCCUPATION_TYPE, dtype: int64

In [4]:
# GET DUMMIES
dummies = pd.get_dummies (df['OCCUPATION_TYPE'], dummy_na=True)
# Do not forget to make the NAs explicitly as a dummy variable, you may forget it which will become
# the reference level in your regression - that us why I used dummy_na = True 
dummies.head()

,Accountants,Cleaning staff,Cooking staff,Core staff,Drivers,HR staff,High skill tech staff,IT staff,Laborers,Low-skill Laborers,Managers,Medicine staff,Private service staff,Realty agents,Sales staff,Secretaries,Security staff,Waiters/barmen staff,NaN
0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
4,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [5]:
df2 = pd.concat([df,dummies], axis=1)
df2 = df2.drop(['Laborers'], axis=1)
# drop a level to avoid multicollinearity in a regression, in other words if we are running
# a regression model, we need to drop the variable that shows multicollinearity 
# with other variavles, however if we are runnig any tree-based allgorithms, we do not need to drop the variable.
# ALso when we drop a variavle in a regression model , it becames the base level that other variables will reference to, 
# so if the count of droped variable is too few, it is not going to be the best level that other variable reference to

df2.head()

,OCCUPATION_TYPE,OWN_CAR_AGE,TARGET,Accountants,Cleaning staff,Cooking staff,Core staff,Drivers,HR staff,High skill tech staff,IT staff,Low-skill Laborers,Managers,Medicine staff,Private service staff,Realty agents,Sales staff,Secretaries,Security staff,Waiters/barmen staff,NaN
0,Laborers,NaN,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Core staff,NaN,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Laborers,26.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Laborers,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Core staff,NaN,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


OneHotEncoder of Category_encoder

In [9]:
pip install category_encoders 

     |████████████████████████████████| 81kB 3.5MB/s 


In [9]:
import category_encoders as ce

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [10]:
X = df.OCCUPATION_TYPE
ec = ce.OneHotEncoder(cols = 'OCCUPATION_TYPE', use_cat_names=True, handle_unknown='indicator').fit (X)
ec.fit_transform(X)


,OCCUPATION_TYPE_Laborers,OCCUPATION_TYPE_Core staff,OCCUPATION_TYPE_Accountants,OCCUPATION_TYPE_Managers,OCCUPATION_TYPE_nan,OCCUPATION_TYPE_Drivers,OCCUPATION_TYPE_Sales staff,OCCUPATION_TYPE_Cleaning staff,OCCUPATION_TYPE_Cooking staff,OCCUPATION_TYPE_Private service staff,OCCUPATION_TYPE_Medicine staff,OCCUPATION_TYPE_Security staff,OCCUPATION_TYPE_High skill tech staff,OCCUPATION_TYPE_Waiters/barmen staff,OCCUPATION_TYPE_Low-skill Laborers,OCCUPATION_TYPE_Realty agents,OCCUPATION_TYPE_Secretaries,OCCUPATION_TYPE_IT staff,OCCUPATION_TYPE_HR staff,OCCUPATION_TYPE_-1
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307506,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
307507,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
307508,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
307509,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


## (B) WEIGHT OF EVIDENCE (WOE) ENCODING



In [11]:
# Category_Encoders WOE
# Weight of Evidence is widely used technique in credit risk modeling or called the probability of default modeling. 
# The goal of such a transfornation is to get the maximum difference among the binned categories relating to the target variable.
# It counts the number of responders and non-responders in each binned categories, then asigns a numeric value to each of the binned categoriea.
# Especiallu sutitable for Logistic Regression
# Remember to create the 'NOData' value for missing values.
# There is no need to cap or floor the outliers
ec = ce.WOEEncoder()
df.OCCUPATION_TYPE = df.OCCUPATION_TYPE.fillna('NoData')
X = df.OCCUPATION_TYPE
y = df.TARGET
X_WOE = ec.fit(X,y)
X_cleaned = ec.transform(X)
X_cleaned.round(2)

,OCCUPATION_TYPE
0,0.30
1,-0.27
2,0.30
3,0.30
4,-0.27
...,...
307506,0.19
307507,-0.23
307508,-0.28
307509,0.30


In [0]:
# What if the target is a continuous variable? You can split the predictor into 10 or 20
# equal parts, then compute the WOE accordingly.

## (C) TARGET ENCODING

In [16]:
# this method simply replaces eatch category with the mean target value for samples which have that category

from category_encoders import target_encoder as te

df.OCCUPATION_TYPE = df.OCCUPATION_TYPE.fillna('NoData')
X = df.OCCUPATION_TYPE
y= df.TARGET
ec = te.TargetEncoder()
X_TE = ec.fit_transform(X,y)
outf= pd.concat([X,X_TE],axis=1)
outf.columns = ['OCCUPATION_TYPE','mean']
outf.head(10)


,OCCUPATION_TYPE,mean
0,Laborers,0.105788
1,Core staff,0.063040
2,Laborers,0.105788
3,Laborers,0.105788
4,Core staff,0.063040
5,Laborers,0.105788
6,Accountants,0.048303
7,Managers,0.062140
8,NoData,0.065131
9,Laborers,0.105788


In [15]:
# add some noises
import numpy as np
cntrl = 0.3
capped = df.TARGET.mean()* cntrl
num_obs = df.shape[0]
noise = np.random.uniform(0,capped, num_obs)
outf['mean']= outf['mean']+noise
outf.head(10)

,OCCUPATION_TYPE,mean
0,Laborers,0.107692
1,Core staff,0.079081
2,Laborers,0.110327
3,Laborers,0.117187
4,Core staff,0.069542
5,Laborers,0.121125
6,Accountants,0.063276
7,Managers,0.079263
8,NoData,0.074326
9,Laborers,0.119024


In [0]:
# Targeting encoding can be seriously plagued by overfittinf because it uses information about the target, called TARGET LEAKING
# to over come this we can add noise to the new variable
# When we predict new records with our model, we do not need to add noise to the new values

## (D) LEAVE-ONE-OUT (LOO) EMCODING

In [5]:
# leave-one-out encoding produces the mean value of the target value of the rows for the same categorical level, excluding the row itself.
# it leaves the target value of the row itself out- thus this encoding avoids direct leaking-

def LOO(var,target):
    # Get the count and the sum statistics by category
    df['OCCUPATION_TYPE'] = df['OCCUPATION_TYPE'].fillna('NoData')
    h = df[['OCCUPATION_TYPE','TARGET']].groupby('OCCUPATION_TYPE')['TARGET'].agg(['count','sum']).reset_index()
    h.columns = ['OCCUPATION_TYPE','Count','Sum']
    # Append to the data
    df2 = pd.merge(df[[var,target]],h,left_on='OCCUPATION_TYPE',right_on='OCCUPATION_TYPE',how='left')
    # Get the mean excluding the row itself to avoid direct target leakage
    df2[var + '_LOO'] = ((df2['Sum'] - df2[target])/(df2['Count'] - 1)).round(2)
    df2 = df2.drop([target,'Count','Sum'],axis=1)
    return(df2)
    
k = LOO('OCCUPATION_TYPE','TARGET')
k.head()

,OCCUPATION_TYPE,OCCUPATION_TYPE_LOO
0,Laborers,0.11
1,Core staff,0.06
2,Laborers,0.11
3,Laborers,0.11
4,Core staff,0.06


## (E) ORDINAL ENCODING

In [0]:
# Ordinal encoding uses additional information to convert the category variable to numeric.
# The numeric differences matter. So we need to consider if the numbers make
# sense and your interpretation is consistent.

## (F) HASHING ENCODING

In [11]:
# If you use one-hot encoding to convert a high-cardinality categorical variable (such as
# zip codes), we will end up with a gigantic sparse matrix. Such sparse matrix is
# extremely inefficient and will result in model instability. The hashing encoding may be
# a better solution. Hashing encoding gains its popularity in the discussions of Kaggle
# competitions. It is similar to one-hot encoding but projects to much less number of
# dimensions. Hashing encoding uses a hash function to convert N levels to M ≤ N
# columns.

from category_encoders import hashing as hs
df['OCCUPATION_TYPE'] = df['OCCUPATION_TYPE'].fillna('NoData')
X = df['OCCUPATION_TYPE'] 
ec = hs.HashingEncoder(n_components=4,hash_method='md5')
X_HASH = ec.fit_transform(X)
outf = pd.concat([X,X_HASH],axis=1)
outf.head()

,OCCUPATION_TYPE,col_0,col_1,col_2,col_3
0,Laborers,0,1,0,0
1,Core staff,0,0,0,1
2,Laborers,0,1,0,0
3,Laborers,0,1,0,0
4,Core staff,0,0,0,1


The disadvantage of the Hashing encoding is the lack of good business
interpretation. Why do “Managers”, “Drivers”, …, “HR staff” all go to the same
group?
The process can be time-consuming.
If we want to have a more interpretable result, we can group categorical levels
manually